In [1]:
import sys
sys.path.append('../src')

In [2]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
from pathlib import Path

from covid import daily_snapshot
import population as pop
#import requests

In [3]:
pd.set_option('max_rows', 300) 

In [4]:
def county_populations_for(state: str) -> pd.DataFrame:
    data = Path('../data/counties')
    state_pop_path = data / Path(state.replace(' ', '').lower() + '.csv')
    state_pop = pd.read_csv(state_pop_path)
    return state_pop

In [5]:
def f(state, date, sort_by, ascending):
    daily = daily_snapshot(datetime.strftime(date, '%m-%d-%Y'))
#     state_covid = pd.read_csv('../data/us-states-only.csv')
    county_pop = county_populations_for(state)
    state_with_population = pd.merge(daily, county_pop,
         how='outer',
         left_on='Admin2', right_on='county')

    state_with_population['fraction_confirmed'] = state_with_population['Confirmed'] / state_with_population['pop2019']   * 1000.0
    state_with_population['deaths']             = state_with_population['Deaths']    / state_with_population['pop2019']   * 1000.0
    state_with_population['death_rate?']        = state_with_population['Deaths']    / state_with_population['Confirmed'] * 1000.0

    # sort_by = input("column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate?")
    print(state_with_population.sort_values(by=sort_by, ascending=False))


In [6]:
cp = county_populations_for('wyoming')

In [7]:
cp.head()

,Rank,county,pop2019
0,1,Laramie,97692
1,2,Natrona,80610
2,3,Campbell,47708
3,4,Sweetwater,44117
4,5,Fremont,40076


In [8]:
state_covid = pd.read_csv('../data/us-states-only.csv')

In [9]:
state_covid.head(2)

,state
0,Alabama
1,Alaska


In [10]:
states = list(pd.read_csv('../data/us-states-only.csv').state)

state = widgets.Select(
    options=states,
    #value='Alabama',
    rows=10,
    description='State:',
    disabled=False
    )

from datetime import datetime, timedelta
date = widgets.DatePicker(description='Date',
                          value=datetime.today() - timedelta(1),
                          disabled=False
                          )

sort_by = widgets.Select(
    options=['Confirmed', 
             'Deaths', 
             'county', 
             'pop2019', 
             'fraction_confirmed', 
             'deaths', 
             'death_rate?'],
    value='fraction_confirmed',
    rows=7,
    description='sort by:',
    disabled=False
    )

ascending = widgets.Checkbox(
    value=True,
    description='Sort Ascending',
    disabled=False,
    indent=False
)

ui = widgets.HBox([state, date, sort_by, ascending])



out = widgets.interactive_output(f, {'state': state, 
                                     'date': date, 
                                     'sort_by': sort_by,
                                     'ascending': ascending})

display(ui, out)

      Unnamed: 0      Admin2 Province_State  Country_Region  \
1396         NaN      Marion        Indiana              US   
573          NaN      Dallas          Texas              US   
1532         NaN      Monroe       New York              US   
1534         NaN      Monroe   Pennsylvania              US   
772          NaN    Franklin           Ohio              US   
...          ...         ...            ...             ...   
2760         NaN         NaN            NaN  Western Sahara   
2761         NaN         NaN            NaN          Zambia   
2762         NaN         NaN            NaN        Zimbabwe   
2763         NaN  unassigned        Wyoming              US   
2764         NaN         NaN            NaN             NaN   

              Last_Update        Lat      Long_  Confirmed  Deaths  Recovered  \
1396  2020-04-05 23:06:45  39.781636 -86.138263     1760.0    37.0        0.0   
573   2020-04-05 23:06:45  32.766706 -96.777961     1112.0    18.0        0.0   


Output()